# Partie II : Modèle (schéma) commun d'utilisation des Hugging Face Transformers

Ce Notebook montre un schéma "classique d'utilisation des Transformers, en utilisant l'exemple de l'analyse de sentiment.

Tout d'abord, trouvez un modèle sur [the hub] (https://huggingface.co/models). Tout le monde peut télécharger son modèle pour que d'autres personnes puissent l'utiliser. 

Ensuite, deux objets doivent être initialisés - un **tokenizer** et un **model**

* Le tokenizer convertit les chaînes de caractères en listes d'identifiants de vocabulaire dont le modèle a besoin.
* Le modèle prend les identifiants de vocabulaire et produit une prédiction.

Installez la bibliothèque 🤗 *Transformers* pour exécuter ce *notebook*.

In [ ]:
!pip install transformers
#[sentencepiece]

## 0. Rappel pipelines 
Dans la première partie de ce Notebook nous avons utilisé un pipeline end-to-end (une ligne de code).
Ce pipeline est en fait composé de trois étapes (voir ci dessous)

In [ ]:
# end-to-end pipeline.

from transformers import pipeline

classifier = pipeline("sentiment-analysis", model="tblard/tf-allocine")
classifier(
    ["J'ai attendu un cours d'HuggingFace toute ma vie.",
     "Je déteste tellement ça !"]
)

## 1. Schéma commun utilisation Hugging Face

Ce pipeline regroupe trois étapes : le prétraitement (le **`tokenizer`**), le passage des entrées dans le **`modèle`** et le **`post-traitement`**.
<figure>
    <img src="./images/full_nlp_pipeline.svg"  style="width:6000px;height:250px;" >
</figure>

###  Etape 1 : Prétraitement - Tokenizer
Les transformers ne peuvent pas traiter directement le texte brut, donc la première étape de notre pipeline est de convertir les entrées textuelles en identifiants (id) afin que le modèle puisse les comprendre. Pour ce faire, nous utilisons un tokenizer, qui sera responsable de :
- diviser l’entrée en mots, sous-mots, ou symboles (comme la ponctuation) qui sont appelés tokens,
- associer chaque token à un nombre entier,
- ajouter des entrées supplémentaires qui peuvent être utiles au modèle (`padding`).

Key features:
- Supports different tokenization techniques like Byte-Pair Encoding (BPE), WordPiece,
and SentencePiece.
- Tokenization happens quickly with parallelization support.
- Handles special tokens like [CLS], [SEP], and padding/truncation automatically.
- Easily load pre-trained tokenizers with AutoTokenizer

In [ ]:
from transformers import AutoTokenizer

#checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
checkpoint = "distilbert/distilbert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.",
    # J'ai attendu un cours de HuggingFace toute ma vie.
    "I hate this so much!",  # Je déteste tellement ça !
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")
print(inputs)

On peut appelerles tokenizers de différentes manières. 
Soit en précisant le nom du modèle pré-entrainé, par exemple `BertTokenizer`, soit en utilisant la classe `AutoTokenizer` qui récupère la classe de `tokenizer` appropriée dans la bibliothèque basée sur le nom du checkpoint.


In [ ]:
from transformers import BertTokenizer, DistilBertTokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")
tokenizer = DistilBertTokenizer.from_pretrained("distilbert/distilbert-base-cased")  
tokenizer=AutoTokenizer.from_pretrained("gpt2")
    
print (tokenizer)

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
print (tokenizer)

In [ ]:
#tokens_inputs=tokenizer("Using a Transformer network is simple")

tokens_inputs=tokenizer("Using a Transformer network is simple",return_tensors="pt")

print("Vanilla Tokenization")
print(tokens_inputs)
print()

# Two ways to access:
print(tokens_inputs.input_ids)
print(tokens_inputs["input_ids"])

La classe `tokenizer ()` peut être utilisée directement, elle s'occupe d'ffectuer toues les opéraions nécessaires pour la construction d'une entrée adéquate au tranformer. Mais, on peut aussi déclider les différentes opérations.  

| Méthode                                       | Retourne                                       | Spécificités                                                           |       
|----------------------------------------------|-----------------------------------------------|------------------------------------------------------------------------|
| `tokenizer(text)`                            | `Dict` (`input_ids`, `attention_mask`, ...)   | Ajoute les **tokens spéciaux** ; supporte `return_tensors="pt"`        |
| `tokenizer.encode(text)`                     | `List[int]` (IDs des tokens)                  | Peut ajouter les tokens spéciaux (`add_special_tokens=True`)          | 
| `tokenizer.encode_plus(text)`                | `Dict` comme `tokenizer(text)`                | Permet `text_pair`, padding, truncation                              | 
| `tokenizer.batch_encode_plus([text1, ...])`  | `List[Dict]`                                  | Pour plusieurs textes (batch), avec padding et autres options         | 
| `tokenizer.tokenize(text)`                   | `List[str]` (tokens lisibles)                 | Affiche les sous-mots BPE / SentencePiece                             | 
| `tokenizer.convert_tokens_to_ids(tokens)`    | `List[int]`                                   | Convertit tokens → IDs                                                | 
| `tokenizer.convert_ids_to_tokens(ids)`       | `List[str]`                                   | Convertit IDs → tokens lisibles                                       | 
| `tokenizer.decode(ids)`                      | `str` (texte lisible)                         | Recompose le texte complet depuis les IDs                             | 



In [ ]:
inputs_str="Using a Transformer network is simple"
print("List tokens ID :", tokenizer.encode(inputs_str))
print("List tokens complets:", tokenizer.encode_plus(inputs_str))

tokens=tokenizer.tokenize(inputs_str)
print("List tokens  :", tokens)

token_ids=tokenizer.convert_tokens_to_ids(tokens)
print("Liste  des tokens Ids :",token_ids)

print("Id to tokens :", tokenizer.convert_ids_to_tokens(token_ids))
print("Id to texte originel :", tokenizer.decode(token_ids))

### Traiter plusieurs séquences de longueurs différentes

In [ ]:
# On peut faire passer plusieurs séquences. 
#si on souhaite récupérer un tenseur on doit utiliser un pading 
#pour que les vecteurs de chaque séquence (texte) ait la même dimension.
# teste avec et sans padding ?

# Si tokenizer de GPT2 :  Set a padding token (use the EOS token for GPT-2)
#tokenizer.pad_token = tokenizer.eos_token

model_inputs = tokenizer(["Hugging Face Transformers is great!",
                         "The quick brown fox jumps over the lazy dog.",
                         "Then the dog got up and ran away because she didn't like foxes.",
                         ],
                         return_tensors="pt", #tf : si tensorflow, ou np: Numpy
                         padding=True,
                         truncation=True
                        )
print(f"Pad token: {tokenizer.pad_token} | Pad token id: {tokenizer.pad_token_id}")
print("Padding:")
print(model_inputs)


### Etape 2 : Passage au modèle- Choix du modèle de Transformer
Il existe de nombreuses architectures différentes disponibles dans la bibliothèque 🤗 Transformers, chacune étant conçue pour prendre en charge d’une tâche spécifique. En voici une liste non exhaustive :

- *Model (récupérer les états cachés : modèle de base)
- *ForCausalLM
- *ForMaskedLM
- *ForMultipleChoice
- *ForQuestionAnswering
- *ForSequenceClassification
- *ForTokenClassification
- et autres 🤗
  
 `*` peut être  `Auto` (par exemple `AutoModel`),  ou un modèle spécifique (e.g. `Distil`Bert)

Il existe 3 classes de modèles 
- Encoders (e.g. BERT)
- Decoders (e.g. GPT2)
- Encoder-Decoder  (e.g. BART or T5)
  
Nous pouvons télécharger notre modèle pré-entraîné de la même manière que nous l’avons fait avec notre tokenizer. Transformers fournit une classe AutoModel qui possède également une méthode from_pretrained().

Attention, les `*Model' sont des modèles de base sans post-training (finetuning). Les autres modèles de type `ForSequenceClassification`, '`For...` PEUVENT AVOIR ETE Fietuné Mais ce n'est pas toujours le cas. 

| Classe de modèle                          | Exemple(s) de modèle                                | Fine-tuné ? | Tâche ciblée                          |
|-------------------------------------------|-----------------------------------------------------|-------------|----------------------------------------|
| `AutoModel`                               | `bert-base-uncased`, `roberta-base`                  | Non      | Représentation de texte (pré-entraînement) |
| `AutoModelForCausalLM`                    | `gpt2`, `llama-7b-hf`                                |  Non      | Modèle génératif (langage auto-régressif) |
| `AutoModelForMaskedLM`                    | `bert-base-uncased`, `distilbert-base-uncased`       |  Non      | Prédiction de tokens masqués (MLM)     |
| `AutoModelForSequenceClassification`      | `distilbert-base-uncased-finetuned-sst-2-english`     | Oui      | Classification de texte                |
| `AutoModelForTokenClassification`         | `dbmdz/bert-large-cased-finetuned-conll03-english`    | Oui      | Reconnaissance d'entités nommées (NER) |
| `AutoModelForQuestionAnswering`           | `distilbert-base-uncased-distilled-squad`              | Oui      | Question Answering (Q&A)               |
| `AutoModelForMultipleChoice`              | `bert-base-uncased`                                    |  Non\*    | Doit être fine-tuné manuellement       |
| `AutoModelForNextSentencePrediction`      | `bert-base-uncased`                                     | Non      | Tâche de pré-entraînement NSP          |



In [126]:
from transformers import AutoModelForSequenceClassification, DistilBertForSequenceClassification, DistilBertModel

print('Loading base model')
checkpoint='distilbert-base-cased'
#checkpoint = "bert-base-uncased"

input_str = "Using a Transformer network is simpled"
# le modèle de base sans aucune tâche 
#base_model = DistilBertModel.from_pretrained(checkpoint)
# Modèles préparée pour la tâche de classification mais pas finetunée
print("Loading classification model from base model's checkpoint")
#model = DistilBertForSequenceClassification.from_pretrained(checkpoint)


model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

print(model.config.num_labels)


Loading base model
Loading classification model from base model's checkpoint


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


2


In [131]:
print(model.parameters)
print("Number of parameters :", model.num_parameters() / 1_000_000)

<bound method Module.parameters of DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropou

In [ ]:
from transformers import AutoConfig

config = AutoConfig.from_pretrained(checkpoint)
print(config)


In [ ]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained(checkpoint)  
model = AutoModel.from_pretrained(checkpoint)

input_str = "Using a Transformer network is simpled"

# Tokenize input string
inputs = tokenizer(input_str, return_tensors="pt")

# Pass to model
outputs = model(**inputs)

# Check output shape # Ecriture pytorch
print(outputs.last_hidden_state.shape)


Vous pouvez accéder aux éléments par attributs (comme nous l’avons fait), par clé (outputs["last_hidden_state"]), ou même par l’index si vous savez exactement où se trouve la chose que vous cherchez (outputs[0]).

In [ ]:
outputs["last_hidden_state"]

#### Choix d'un modèle finetuné 

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

text = "I love Hugging Face!"

inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)

# Supprimer token_type_ids (non supporté par DistilBERT)
#inputs.pop("token_type_ids", None)

# Appel du modèle
outputs = model(**inputs)

# Accès aux logits
print(outputs.logits)


In [ ]:
# dans le cas
print(outputs.logits.shape)

### Post-traitement de la sortie

In [ ]:
print(outputs.logits)

In [ ]:
import torch

predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)

Maintenant nous pouvons voir que le modèle a prédit [0.0402, 0.9598] . Ce sont des scores de probabilité reconnaissables.

Pour obtenir les étiquettes correspondant à chaque position, nous pouvons inspecter l’attribut id2label de la configuration du modèle (plus de détails dans la section suivante) :

In [ ]:
model.config.id2label

### Génération de textes

In [125]:
from transformers import AutoTokenizer, AutoModelForCausalLM
checkpoint="gpt2"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)  
model = AutoModelForCausalLM.from_pretrained(checkpoint)

input_str = "Using a Transformer network is simpled"

# Tokenize input string
inputs = tokenizer(input_str, return_tensors="pt")

# Pass to model
#outputs = model.generate(inputs.input_ids,max_length=10)

outputs =model.generate(
    inputs.input_ids,
    max_length=20,
    pad_token_id=tokenizer.eos_token_id,   
    attention_mask=inputs.attention_mask   
)

# Decode
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Using a Transformer network is simpled by the fact that the network is connected to the network.
